In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
#import uproot
import pandas as pd
import statistics as st
import os
from numba import njit
from numba import jit
import time
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
from loguru import logger
from datetime import datetime

In [3]:
# warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
# warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
# warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# for arg in sys.argv:
#     run_num=str(arg).zfill(5)
#     # print(run_num)

# chan_enab = int(sys.argv[-1])
# run_start=str(sys.argv[1]).zfill(5)
# run_end=str(sys.argv[2]).zfill(5)
# run_num=str(sys.argv[3]).zfill(5)

run_num = '12035'
runs_folder = 'runs12034-12363/'
os.chdir('F:/LANL/')
datadir = 'sample_data/'
uniquefolder = 'debug_sample/'+runs_folder
SFNormFile = 'SF_Norm_files/'+runs_folder+run_num
statefileloc = 'F:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'

# print(os.getcwd())

# os.chdir('F:/LANL/')
# datadir = 'D:/LANSCE_FP12_2023/data/' ## add directory of hard drive
# uniquefolder = "runs" + str(run_start) + "-" + str(run_end) +"/"
# SFNormFile = 'SF_Norm_files/'+uniquefolder+run_num

print('processing data: ' + uniquefolder + '/run' + run_num)

# statefileloc = 'F:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'
# processedpulsefolder = '/processed_data/'+uniquefolder+'pulses_added_D/'
# processedasymfolder = '/processed_data/'+uniquefolder+'asym_D/'
# AddedPulseSavename = processedpulsefolder+run_num+'_pulsesadded_D'
# AsymSavename = processedasymfolder+run_num+'_asym_D'
# ONSavename = processedasymfolder+run_num+'_ON_D'
# OFFSavename = processedasymfolder+run_num+'_OFF_D'
# logger.add("F:/LANL/processed_data/" + uniquefolder + '0_ErrorLog_'+run_start+'_'+run_end+'_D.txt', delay = False)
# print('saving processed data to ' + AsymSavename)

## cannot handle all 24 detectors at once, memory issue... can look into np.empty and deleting variables if needed<br>
# chan_enab = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]) ## all
chan_enab = np.array([0,1,2,3,4,5,6,7,8,9,10,24]) ## downstream
# chan_enab = np.array([12,13,14,15,16,17,18,19,20,21,22,23,24]) ## upstream

# if not os.path.exists(os.getcwd()+processedpulsefolder) or not os.path.exists(os.getcwd()+processedasymfolder):
#     # Create the directory
#     os.makedirs(os.getcwd()+processedpulsefolder)
#     os.makedirs(os.getcwd()+processedasymfolder)
#     print("Directory created successfully")
# else:
#     pass

start = time.time()
fullstart = time.time()

# read_data = np.array([])
# fileLength = np.array([])
read_data = []
fileLength = []

def open_file():
    for el in chan_enab:
        # f = open(datadir + folder + 'run' + run_num + "_ch" + str(el) + ".bin", 'rb')
        f = open(datadir+uniquefolder + 'run' + str(run_num) + "_ch" +str(el) + ".bin", 'rb')
        read_data.append(np.fromfile(file=f, dtype=np.uint16))
        f.close()
        fileLength.append(len(read_data[-1]))
    return read_data, fileLength

open_file()

fileLength = np.asarray(fileLength)
read_data = np.asarray(read_data) ## in detector's case, all are the same size samples, so can do read_data as np array

if chan_enab[-1] != 24:
    emessage = ('last channel is not 6Li detector')
    logger.error(run_num + emessage)
    raise Exception(emessage)

end = time.time()
# print('file open time: ' + str(end-start))            

print("Channel is " + str(chan_enab))
end = time.time()
# print(end-start)
# print(read_data)

processing data: debug_sample/runs12034-12363//run12035
Channel is [ 0  1  2  3  4  5  6  7  8  9 10 24]


Store the big header for each channel in arrays

In [4]:
# In[2]:

BoardID = []
recordLength = []
numSamples = []
eventCounter = []
decFactor = []
chanDec = []
postTrig = []
groupStart = []
groupEnd = []
timestamp= []
sizeFirstEvent = []
TTT = []

targetDict = {0: "La", 1: "Tb2O3", 2: "Yb2O3", 3: "Sm2O3", 4: "Er2O3", 5: "Ho2O3", 6: "other"}
foilDict = {0: "TBD", 1: "TBD", 2: "TBD", 3: "TBD", 4: "TBD", 5: "TBD", 6: "other"}

target=(read_data[0][5]&0x00F0)>>4
foil=read_data[0][5]&0x000F
targetFlag = read_data[0][5]>>8&1
foilFlag = read_data[0][5]>>9&1
spinFiltFlag = read_data[0][5]>>10&1
spinFlipFlag = read_data[0][5]>>11&1
shutterFlag = read_data[0][5]>>12&1
facilityTrigFlag = read_data[0][5]>>13&1

if targetFlag:
    target=targetDict[(read_data[0][5]&0x00F0)>>4]
else:
    target = "empty"
if foilFlag:
    foil=foilDict[read_data[0][5]&0x000F]
else:
    foil = "empty"
for i in range(0,len(chan_enab)):
    BoardID.append(read_data[i][9]>>8)
    recordLength.append(((read_data[i][9]&0x00FF)<<16)+read_data[i][8])
    numSamples.append(((read_data[i][11]&0x00FF)<<16)+read_data[i][10])
    eventCounter.append(read_data[i][6]+(read_data[i][7]<<16))
    BoardID.append(read_data[i][9]>>8)  
    decFactor.append(read_data[i][11]>>8)
    chanDec.append(read_data[i][13]>>8)
    postTrig.append(read_data[i][15]>>8)
    groupStart.append(((read_data[i][13]&0x00FF)<<16)+read_data[i][12])
    groupEnd.append(((read_data[i][15]&0x00FF)<<16)+read_data[i][14])
    
    timestamp.append(read_data[i][16]+(read_data[i][17]<<16)+(read_data[i][18]<<32)+(read_data[i][19]<<40))  
    sizeFirstEvent.append(read_data[i][0]+(read_data[i][1]<<16))
    TTT.append(read_data[i][2]+(read_data[i][3]<<16)+(read_data[i][4]<<32))
    
#     print("For channel " + str(chan_enab[i]) + ", BoardID is " + str(BoardID[i])
#           + "; record length is " + str(recordLength[i]) + "; num Samples is " 
#           + str(numSamples[i]) + "; event counter is " + str(eventCounter[i]) + "; dec factor is " + str(decFactor[i]) + "; chan dec is " 
#           + str(chanDec[i]) + "; postTrig is " + str(postTrig[i]) + "; group start is " + str(groupStart[i]) + "; group end is " + str(groupEnd[i])
#           + "; epoch time is " + str(timestamp[i]) +  "; first event size is " + str(sizeFirstEvent[i]) + "; and ETTT is " + str(TTT[i]) + "\n")

BoardID = np.asarray(BoardID) 
recordLength = np.asarray(recordLength)
numSamples = np.asarray(numSamples)
eventCounter = np.asarray(eventCounter)
decFactor = np.asarray(decFactor)
chanDec = np.asarray(chanDec)
postTrig = np.asarray(postTrig)
groupStart = np.asarray(groupStart)
groupEnd = np.asarray(groupEnd)
timestamp = np.asarray(timestamp)
sizeFirstEvent = np.asarray(sizeFirstEvent)
TTT = np.asarray(TTT)

print("Target is " + target)
# print("Foil is " + foil)
# print("Shutter is open: " + str(bool(shutterFlag)))
# print("Facility t0 is on: " + str(bool(facilityTrigFlag)))
# print("Spin flipper is on: " + str(bool(spinFlipFlag)))
# print("Spin filter is on: " + str(bool(spinFiltFlag)))
# print("Target is present: " + str(bool(targetFlag)))
# print("Foil is present: " + str(bool(foilFlag)))

Target is La


Determine the time axis for each channel

In [5]:
# In[3]:

preTime = []
startTime = []
endTime = []
resolution = []
xs = [] 

for i in range(0,len(chan_enab)):
    preTime.append((100-postTrig[i])*recordLength[i]/100)
    startTime.append((-1*preTime[i]*16*decFactor[i] + groupStart[i]*16*decFactor[i]))
    endTime.append((-1*preTime[i]*16*decFactor[i] + groupEnd[i]*16*decFactor[i]))
    resolution.append(16*chanDec[i]*decFactor[i])
#     print("Pretime for channel", chan_enab[i],"is " + str(preTime[i]) + "; start time is " + str(startTime[i]) + "; end time is " + str(endTime[i]) 
#           + "; resolution is " + str(resolution[i]) + "ns")
    xs.append(np.arange(startTime[i],(numSamples[i])*resolution[i]+startTime[i], resolution[i]))

# np.asarray(preTime)
# np.asarray(startTime)
# np.asarray(endTime)
# np.asarray(resolution)

xs = np.asarray(xs) ## can convert xs to np array here because all detectors same numsamples

In [6]:
# In[16]:

start=time.time()

@njit
def dataread(data, channels, fileLen, numSamps):
    numRuns = int((fileLen[0]-20-numSamps[0])/(numSamps[0]+6)+1)
    ys_arr = np.zeros((len(channels), numRuns,numSamps[0]), dtype=np.uint16)
    ETTT_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    eventcount_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    for i in range(0,len(channels)):
        eventCount = 0
        byteCounter = 0
            #byte counter is really 2bytecounter, lol
        while byteCounter < fileLen[i]:
            if byteCounter == 0:
                ETTT_arr[i]=TTT[i]
                #ETTT_arr[i].append(TTT[i])
                eventcount_arr[i]=(eventCounter[i])
                byteCounter = 20
            else:
                ETTT_arr[i]=(data[i][byteCounter]+(data[i][byteCounter+1]<<16)+(data[i][byteCounter+2]<<32))
                eventcount_arr[i]=(data[i][byteCounter+4]+(data[i][byteCounter+5]<<16))
                byteCounter += 6
            for j in range(0, numSamps[i]):
                #if j == 0:
                    #ys_arr[i].append([])
                #print(byteCounter)
                ys_arr[i][eventCount][j]=data[i][byteCounter]
                byteCounter += 1
            eventCount += 1
    return ys_arr, ETTT_arr, eventcount_arr

# start=time.time()
# ys_arrHe, ETTT_arrHe, eventcount_arrHe  = dataread(read_data, [25], fileLength, numSamples) ##hardcoded channel 25 for He
ys_arr, ETTT_arr, eventcount_arr  = dataread(read_data, chan_enab, fileLength, numSamples) ##hardcoded channels for coils

end = time.time()
print('dataread from binary time: ' + str(end-start))

dataread from binary time: 1.9272782802581787


In [7]:
# In[4]:

timeDif=[]
for i in range(0,len(chan_enab)):
    timeDif.append([])
    for j in range(len(ETTT_arr[i])-1):
        timeDif[i].append((ETTT_arr[i][j+1]-ETTT_arr[i][j])*8)
#     print("Min time difference for channel", chan_enab[i], "is", min(timeDif[i]), "ns")
#     print("Max time difference for channel", chan_enab[i], "is", max(timeDif[i]), "ns \n")
#print(timeDif)

In [8]:
# In[6]:

# Load in SF and He normalization information ##
try:
    df_SF = pd.read_hdf(SFNormFile + '.h5', key='df_0')
    df_HE = pd.read_hdf(SFNormFile + '.h5', key='df_1')
except Exception as e:
    logger.error(run_num + ' failed during SFNormFile load')
    logger.exception(e)

SF_Sort_arr = df_SF[['nicknames', 'transition_locations']].to_numpy().T
He_Norm_arr = df_HE[['pulse', 'norms']].to_numpy().T

NormFactor = 1000000  ## He integrals are huge, this normalizes all of those by a constant value for ease of use
HeNorms= (He_Norm_arr[1])/NormFactor

print(SF_Sort_arr)
print(He_Norm_arr[1]/NormFactor)

[[6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1
  2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5
  6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1
  2 3 4]
 [39 84 129 174 219 264 309 354 399 444 489 534 579 624 669 714 759 804
  849 894 939 984 1029 1074 1119 1164 1209 1254 1299 1344 1389 1434 1479
  1524 1569 1614 1659 1704 1749 1794 1839 1884 1929 1974 2019 2064 2109
  2154 2199 2244 2289 2334 2379 2424 2469 2514 2559 2604 2649 2694 2739
  2784 2829 2874 2919 2964 3009 3054 3099 3144 3189 3234 3279 3324 3369
  3414 3459 3504 3549 3594 3639 3684 3729 3774 3819 3864 3909 3954 3999
  4044 4089 4134 4179 4224 4269 4314 4359 4404 4449 4494 4539 4584 4629
  4674 4719 4764 4809 4854 4899 4944 4989]]
[6.75914521 6.7224181  6.82175878 ... 6.42678369 6.47034032 6.41850394]


In [9]:
# In[18]:
# basesub and plotting ##

baseL = 0
baseR = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])
numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)
legend =  ['NaI', 'R']

start = time.time()

# s = 20 ## pulse to look at
# t=s+1

#  dont know why this is so slow ##

def plotter(ys, xs, baseR, numpoints):
    tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
    for i in range((len(ys))):
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseL:baseR]))
        for j in range(s, t): ## plot only interested pulses
            plt.plot(xs[i], tempys_basesub[i][j]) #label=legend[i]) #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
            plt.axvline(xs[0][baseL], ls = '--')
            plt.axvline(xs[0][baseR], ls = '--')
            #plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
            plt.axvline(xs[0][baseR+5], ls = '--', c ='r') ## BaseR+5 line marks the beginning of the integral, until the end of samples.
#             plt.title('SF state transition' + transitions[p]) 
#             plt.xlabel("time from trigger (ns)")
#             plt.ylabel("ADC")
            plt.legend()
            
# plotter(ys_arr[9:], xs[9:], baseR, numSamples) ##plot coils

ys_basesub = np.zeros((len(ys_arr), numRuns,numSamples[0]), dtype=np.float64)
# ys_basesub_norm = np.zeros((len(ys_arr), numRuns,numSamples[0]), dtype=np.float64)

# @njit ## jit is faster for large # channels, slower for small # channels
# def basesub(ys, baseRight, numpoints): 
#     tempys_basesub = np.zeros((numRuns,numpoints[0]), dtype=np.float64)
#     for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
#         tempys_basesub[pulse]=np.subtract(ys[pulse], np.mean(ys[pulse][baseL:baseRight]))
#     return tempys_basesub

@njit ## jit is faster for large # channels, slower for small # channels
def basesub_norm(ys, baseRight, numpoints): 
    tempys_basesub = np.zeros((numRuns,numpoints[0]), dtype=np.float64)
    for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
        tempys_basesub[pulse]=np.subtract(ys[pulse], np.mean(ys[pulse][baseL:baseRight]))
        tempys_basesub[pulse]=tempys_basesub[pulse]/HeNorms[pulse] 
    return tempys_basesub

#  got rid of sums here, should be done after aligning and cutting
#  got rid of xs in basesub, don't think we need them as an input 06.10.24

# for i in range(len(ys_basesub)): ## feeding y arrays into function 1 channel at  a time is faster than all at once
#     ys_basesub[i] = basesub(ys_arr[i], baseR, numSamples)
for i in range(len(ys_basesub)): ## feeding y arrays into function 1 channel at  a time is faster than all at once
    ys_basesub[i] = basesub_norm(ys_arr[i], baseR, numSamples)

ys_basesub[-1] = ys_basesub[-1]*-1 ## invert 6Li to positive signal. Comment out if not using
# ys_basesub_norm[-1] = ys_basesub_norm[-1]*-1 ## invert 6Li to positive signal. Comment out if not using

end = time.time()
print('plotting and/or base subtraction time: ' + str(end-start))            

plotting and/or base subtraction time: 3.5454230308532715


In [10]:
# In[10]:

# use 6Li t0 for all instead of for themselves individually ##

start = time.time()

NaIthresh=2000
Li6thresh=1000

threshold_array = (np.full(len(ys_basesub), NaIthresh))
threshold_array[-1] = Li6thresh

# njit ## numba does not support reversed, but this could be changed if it's slow
def find_offset(ys, thresharr):
    xCrosses = np.zeros((len(ys), numRuns)) #outer array is crossing arrays for given channel, inner array is crossing for each event
    offset = np.zeros((len(ys), numRuns), dtype=np.int32) ##offset in bins for each channel, each pulse
    modeCrosses = np.zeros((len(ys)), dtype=np.float64)
    for i in reversed(range(len(ys))):
        #xValues.append([])
        for p in range(len(ys[i])):
            xing = np.argmax(ys[i][p] > thresharr[i])
            #print(xing)
            xCrosses[i][p] = xing
        modeCrosses[i] = (st.mode(xCrosses[i])) #find the most typical crossing value for each channel
        for p in range(len(xCrosses[i])):
            offset[i][p] = (modeCrosses[-1] - xCrosses[i][p]) ## make sure this is the correct sign!!! 
    if (np.all(xCrosses[-1])) == False:
        emessage = ('ERROR: 6Li threshold was not reached for at least one pulse')
        logger.error(run_num + emessage)
        raise Exception(emessage)
    return offset, xCrosses, modeCrosses
                           
# offset, xCrosses, modeCrosses = find_offset(ys_basesub, threshold_array)

end = time.time()
# print('finding offset time: ' + str(end-start))  

In [12]:
# In[11]:

#  this cell loops through every channel as opposed to inputting all channels at once. 5x faster ##

start = time.time()

# extend all arrays by a value, check that the max number of offset on 6Li is less than that value ##

# extendedRange = 3 ## must be a positive value which to extend ys_arr
# if abs(max(offset[-1], key = abs)) > extendedRange: ## if the max offset of 6Li is >extendedRange, something is wrong
#     emessage = ('ERROR: largest offset greater than extended range')
#     logger.error(run_num + emessage)
#     raise Exception(emessage)

# ys_ext = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+extendedRange*2), dtype=np.float64)
# ys_cut = np.zeros((len(ys_basesub), len(ys_basesub[0]), (len(ys_ext[0][0])-((extendedRange*2)+1)*2)))

# Below commented out for testing without t0 aligning 08.08.24 ##
# try:
#     ys_ext = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+extendedRange*2), dtype=np.float64)
#     ys_cut = np.zeros((len(ys_basesub), len(ys_basesub[0]), (len(ys_ext[0][0])-((extendedRange*2)+1)*2)))
#     xs_cut = np.zeros((len(ys_cut), len(ys_cut[0][0])))
# except Exception as e:<br>
#     logger.error(run_num + ' failed during ys_cut array creation')
#     logger.exception(e)

# cant use jit because np.pad is not supported ##

def align_cut(ys, xs_arr, extendedr):
    tempys_ext = np.zeros((len(ys), len(ys[0])+extendedr*2), dtype=np.float64)
    tempys_cut = np.zeros((len(ys), (len(tempys_ext[0])-((extendedr*2)+1)*2)))
    tempxs_cut = np.zeros(len(tempys_cut[0]))
    for p in range(len(ys)):
        tempys_ext[p] = np.pad(ys[p], extendedr, 'constant', constant_values=(0))
        tempys_ext[p] = np.roll(tempys_ext[p],offset[-1][p]) ## assumes 6Li at -1 position
        tempys_cut[p] = tempys_ext[p][((extendedr*2)+1):-((extendedr*2)+1)].copy() ## cut by 7 (if extRange == 3)
#         tempys_cut[p] = tempys_cut[p]/HeNorms[p] ## normalize by 3He integral
    x_cut_amt = int((len(ys[0]) - len(tempys_cut[0]))/2)
    tempxs_cut = xs_arr[x_cut_amt:-x_cut_amt].copy()
    return tempys_cut, tempxs_cut

# Below commented out for testing without t0 aligning 08.08.24 ##
# try:
#     for i in range(len(ys_basesub)):
#         ys_cut[i], xs_cut[i] = align_cut(ys_basesub[i], xs[i], extendedRange)
# except Exception as e:
#     logger.error(run_num + ' failed aligning and cutting')
#     logger.exception(e)

    
# checkp = 2053
# print(offset[-1][checkp]) ## checking offset for one example checkpulse
# print('original index for checkpulse: '+str(np.argmax(ys_basesub[0][checkp]> 2000))) ## we can follow the index as it changes with extension/cut
# #print('extended range index for checkpulse: '+str(np.argmax(ys_ext[0][checkp]> 2000)))
# print('cut array index for checkpulse: '+str(np.argmax((ys_cut[0][checkp]*HeNorms[checkp])> 2000)))

# del ys_ext ## might help with memory issues
# del ys_basesub

end = time.time()
print('aligning and cutting time: ' + str(end-start))            

aligning and cutting time: 0.0010006427764892578


In [13]:
# In[7]:

def organize_SF(SFsort_info): ## sometimes pulse 0 has the state switch. In that case, need to account by if clauses below
    counter = 0
    seq = 0
    seq_arr = ([[],[],[]])
    smallerseq = []
    smallerstateis = []
    for i in range(len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
        counter = counter+1
        if counter < 8:
            if (SF_Sort_arr[1][i]) == 0: ## catches state switches at pulse 0
                smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
                smallerseq.append(SFsort_info[0][i+1])
                seq = seq+1
                continue
            smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
            smallerseq.append(SFsort_info[0][i+1])
        elif counter == 8:
            if ((SF_Sort_arr[1][i])+45) >= 5000: ## breaks for state switches at pulse 0
                print(((SF_Sort_arr[1][i])+5))
                seq = seq+1
                seq_arr[0].append(seq)
                seq_arr[1].append(smallerseq)   
                seq_arr[2].append(smallerstateis)
                seq_arr[0] = [x-1 for x in seq_arr[0]] ## reset so sequences are 1-14 instead of 2-15
                break
            seq = seq+1 ## otherwise continue regular sorting
            smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
            smallerseq.append(SFsort_info[0][i+1])
            seq_arr[0].append(seq)
            seq_arr[1].append(smallerseq)   
            seq_arr[2].append(smallerstateis)
            smallerseq = []
            smallerstateis = []
            counter  = 0
    return seq_arr

def find_leftover(SFsort_info, seq_arr): ## in case we want to use the other 6 states left over
    left = [[seq_arr[0][-1]+1],[],[]]
    counter = 0
    for i in range((len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))), len(SFsort_info[1])-1):
        counter = counter+1
        if counter < 8:
            left[1].append(SFsort_info[0][i+1])
            left[2].append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
    return left

try:
    sequence = organize_SF(SF_Sort_arr)
    if len(sequence[0]) == 14: ## catches state switches at pulse 0, leftovers are at the end of the regular sequence
        leftovers = [[sequence[0][-1]],[sequence[1][-1]],[sequence[2][-1]]]
        for i in range(len(sequence)):
            sequence[i].pop(-1) ## deletes the leftovers sequence for state switches at pulse 0
    else:
        leftovers = find_leftover(SF_Sort_arr, sequence) ## otherwise can use normal function
except Exception as e:
    logger.error(run_num + ' failed during sequencing')
    logger.exception(e)

print('sequences '+str(sequence[0]))

print(str(len(sequence[0]))+' sequences with sequence order: '+str(sequence[1][0]))
# print(leftovers)

sequences [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
13 sequences with sequence order: [7, 0, 1, 2, 3, 4, 5, 6]


In [14]:
print(sequence)
print()
print(sequence[1][0][0:4])
print(sequence[2][0][0:4])
print()
seq = 0
state = 2
s = sequence[1][seq][state]
p_start = sequence[2][seq][state][0]
p_end = sequence[2][seq][state][1]
print(s)
print(p_start)
print(p_end)
print()

print(range(0, len(sequence[1][0])))
# for seq in range(0, len(sequence[0])): ## for every sequence
# #         print('seq:' +str(SFarr[0][seq]))
#     print('seq:' +str(seq))
#     for state in range(0, len(sequence[1][0])):
#         print (state)
#         if state==0 or state==3 or state==5 or state==6: ## these are ON states
#             print('"state" ' +str(state)) 
#         if state==0 or state==3 or state==5 or state==6: ## these are ON states
#             print('"state" ' +str(state)) 

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [[7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6], [7, 0, 1, 2, 3, 4, 5, 6]], [[[44, 84], [89, 129], [134, 174], [179, 219], [224, 264], [269, 309], [314, 354], [359, 399]], [[404, 444], [449, 489], [494, 534], [539, 579], [584, 624], [629, 669], [674, 714], [719, 759]], [[764, 804], [809, 849], [854, 894], [899, 939], [944, 984], [989, 1029], [1034, 1074], [1079, 1119]], [[1124, 1164], [1169, 1209], [1214, 1254], [1259, 1299], [1304, 1344], [1349, 1389], [1394, 1434], [1439, 1479]], [[1484, 1524], [1529, 1569], [1574, 1614], [1619, 1659], [1664, 1704], [1709, 1749], [1754, 1794], [1799, 1839]], [[1844, 1884], [1889, 1929], [1934, 1974], [1979, 2019], [2024, 2064], [2069, 2109], [211

In [15]:
sequence = np.asarray(sequence, dtype = object)
print(len(sequence[0]))
print(sequence[1][0])
print(type(sequence[1][0]))
print(len(sequence[1][0]))
print(type(len(sequence[1][0])))
print(type(sequence))
type(np.asarray(sequence, dtype = object))
print(type(len(sequence[0])))
print()
a = np.asarray(sequence[1][0])
print(a)
print(type(a))
print(np.where(a==8))
b = np.where(a==6)
print(b)
print(b[0][0])
OFF_1 = 2
print((sequence[1][0][OFF_1]))

13
[7, 0, 1, 2, 3, 4, 5, 6]
<class 'list'>
8
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>

[7 0 1 2 3 4 5 6]
<class 'numpy.ndarray'>
(array([], dtype=int64),)
(array([7], dtype=int64),)
7
1


In [ ]:
a = np.array([0,1,2,3,4,5])
b = np.array([10,10,10,20,20,20])
c = np.add(a,b)
print(c)

In [16]:
# In[46]:

#  add up pulses for their respective state, in each 8 step sequence ##
#  turning into a by-channel function 06.13.24 ##

start = time.time()

legend = ['NaI5 (downstream)','NaI4R (upstream)','6Li']
sequence = np.asarray(sequence, dtype = object)

# added_pulses = np.zeros((len(ys_cut), len(sequence[0]), 8, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 8 stages each works?<br>
# i channels, 13 sequences each, 8 states each sequence, 8992 num points ##

# ON_OFF_sums = np.zeros((len(ys_cut), len(sequence[0]), 2, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 2 for ON or OFF for each sequence
ON_sums = np.zeros((len(ys_basesub), len(sequence[0]), len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for ON
OFF_sums = np.zeros((len(ys_basesub), len(sequence[0]), len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for OFF
ON_minus_sums = np.zeros((len(ys_basesub), len(sequence[0]),2, len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for ON
ON_plus_sums = np.zeros((len(ys_basesub), len(sequence[0]),2, len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for OFF
# ON_minus/plus_sums have their 2 for associated OFF states in the second array

# @njit
def add_pulse(ys, SFarr):
#     tempadded_p = np.zeros((len(SFarr[0]), 8, len(ys[0])), dtype=np.float64)
    temp_ON = np.zeros((len(SFarr[0]), len(ys[0])), dtype=np.float64)
    temp_OFF = np.zeros((len(SFarr[0]), len(ys[0])), dtype=np.float64)
    temp_ON_min = np.zeros((len(SFarr[0]),2, len(ys[0])), dtype=np.float64)
    temp_ON_plu = np.zeros((len(SFarr[0]),2, len(ys[0])), dtype=np.float64)
    for seq in range(0, len(SFarr[0])): ## for every sequence
    #         print('seq:' +str(SFarr[0][seq]))
        print('seq:' +str(seq))
        for state in range(0, len(SFarr[1][0])): ## for every state in the sequence
    #         print("states loop " + str(range(0, len(SFarr[1][0]))[0]) + ' - ' +  str(range(0, len(SFarr[1][0]))[-1]))
            s = SFarr[1][seq][state] ## try this to condense code. Basically, the state currently at
            if s==0 or s==3 or s==5 or s==6: ## these are ON states
                print('ON "s" state '+str(s))
#                 print('"state" ' +str(state) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
                print('sums from '+str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[0]) +
                ' - ' +str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[-1]) + '\n')
                for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                    temp_ON[seq] = np.add(temp_ON[seq],ys[p]) ## start with zeros, add to each iteratively
#                     print(p)
                #         below is for splitting up into ON(-) and ON(+) and their associated OFF states (1 state forward for -, 1 state backward for +)
                if s==0 or s==6: ## these or ON(-) states
                    OFF_i = np.where(np.asarray(SFarr[1][0])==s+1)[0][0] ## find where in sequence state = s(ON-)+1    
    #                 print('ON(-) "s" ' +str(s) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
                    for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_min[seq][0] = np.add(temp_ON_min[seq][0],ys[p]) ## start with zeros, add to each iteratively
    #                 print('OFF state ' + str(SFarr[1][seq][OFF_i]))
                    for p in range((SFarr[2][seq][OFF_i][0]),(SFarr[2][seq][OFF_i][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_min[seq][1] = np.add(temp_ON_min[seq][1],ys[p]) ## start with zeros, add to each iteratively
                if s==3 or s==5: ## these or ON(+) states
                    OFF_i = np.where(np.asarray(SFarr[1][0])==s-1)[0][0] ## find where in sequence state = s(ON+)-1    
    #                 print('ON(+) "s" ' +str(s) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
                    for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_plu[seq][0] = np.add(temp_ON_plu[seq][0],ys[p]) ## start with zeros, add to each iteratively
    #                 print('OFF state ' + str(SFarr[1][seq][OFF_i]))
                    for p in range((SFarr[2][seq][OFF_i][0]),(SFarr[2][seq][OFF_i][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_plu[seq][1] = np.add(temp_ON_plu[seq][1],ys[p]) ## start with zeros, add to each iteratively
            if s==1 or s==2 or s==4 or s==7: ## these are OFF states
                print('OFF "s" state '+str(s))
#                 print('"state" ' +str(state) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
                print('sums from '+str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[0]) +
                ' - ' +str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[-1]) + '\n')
                for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                    temp_OFF[seq] = np.add(temp_OFF[seq],ys[p]) ## start with zeros, add to each iteratively
#                     print(p)
    return temp_ON, temp_OFF, temp_ON_min, temp_ON_plu

for i in range(len(ys_basesub)):
    print('#################### channel: ' + str(i) + ' ##########################')
    ON_sums[i], OFF_sums[i], ON_minus_sums[i], ON_plus_sums[i] = add_pulse(ys_basesub[i], sequence)
    
# for i in range(len(ys_basesub)-12):
#     print('#################### channel: ' + str(i) + ' ##########################')
#     ON_sums[i], OFF_sums[i], ON_minus_sums[i], ON_plus_sums[i] = add_pulse(ys_basesub[i], sequence)

# np.save(os.getcwd() + ONSavename, ON_sums)
# np.save(os.getcwd() + OFFSavename, OFF_sums)
                
# plotting examples ##
# plt.plot(xs_cut[i], added_pulses[0][0][0] , label=legend[0] +', sequence 1 state 1, 40 pulses added')
# plt.plot(xs_cut[i], added_pulses[0][0][1] , label=legend[0] +', sequence 1 state 2, 40 pulses added')
# plt.plot(xs_cut[i], added_pulses[0][1][0] , label=legend[0] +', sequence 2 state 1, 40 pulses added') 
# plt.plot(xs_cut[i], added_pulses[0][1][1] , label=legend[0] +', sequence 2 state 2, 40 pulses added') 
    
# plt.title('Detector signals') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")

# plt.axvline(xs[0][baseL], ls = '--')
# plt.axvline(xs[0][baseR], ls = '--')
# plt.axvline(xs[1][intgrL], ls = '--', c ='g')
# plt.axvline(xs[1][intgrR], ls = '--', c ='g')
# plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
# plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

# plt.legend()
# plt.show()

end = time.time()
print('summing pulses into their states time: ' + str(end-start))  

#################### channel: 0 ##########################
seq:0
OFF "s" state 7
sums from 44 - 83

ON "s" state 0
sums from 89 - 128

OFF "s" state 1
sums from 134 - 173

OFF "s" state 2
sums from 179 - 218

ON "s" state 3
sums from 224 - 263

OFF "s" state 4
sums from 269 - 308

ON "s" state 5
sums from 314 - 353

ON "s" state 6
sums from 359 - 398

seq:1
OFF "s" state 7
sums from 404 - 443

ON "s" state 0
sums from 449 - 488

OFF "s" state 1
sums from 494 - 533

OFF "s" state 2
sums from 539 - 578

ON "s" state 3
sums from 584 - 623

OFF "s" state 4
sums from 629 - 668

ON "s" state 5
sums from 674 - 713

ON "s" state 6
sums from 719 - 758

seq:2
OFF "s" state 7
sums from 764 - 803

ON "s" state 0
sums from 809 - 848

OFF "s" state 1
sums from 854 - 893

OFF "s" state 2
sums from 899 - 938

ON "s" state 3
sums from 944 - 983

OFF "s" state 4
sums from 989 - 1028

ON "s" state 5
sums from 1034 - 1073

ON "s" state 6
sums from 1079 - 1118

seq:3
OFF "s" state 7
sums from 1124 - 1163



OFF "s" state 1
sums from 1574 - 1613

OFF "s" state 2
sums from 1619 - 1658

ON "s" state 3
sums from 1664 - 1703

OFF "s" state 4
sums from 1709 - 1748

ON "s" state 5
sums from 1754 - 1793

ON "s" state 6
sums from 1799 - 1838

seq:5
OFF "s" state 7
sums from 1844 - 1883

ON "s" state 0
sums from 1889 - 1928

OFF "s" state 1
sums from 1934 - 1973

OFF "s" state 2
sums from 1979 - 2018

ON "s" state 3
sums from 2024 - 2063

OFF "s" state 4
sums from 2069 - 2108

ON "s" state 5
sums from 2114 - 2153

ON "s" state 6
sums from 2159 - 2198

seq:6
OFF "s" state 7
sums from 2204 - 2243

ON "s" state 0
sums from 2249 - 2288

OFF "s" state 1
sums from 2294 - 2333

OFF "s" state 2
sums from 2339 - 2378

ON "s" state 3
sums from 2384 - 2423

OFF "s" state 4
sums from 2429 - 2468

ON "s" state 5
sums from 2474 - 2513

ON "s" state 6
sums from 2519 - 2558

seq:7
OFF "s" state 7
sums from 2564 - 2603

ON "s" state 0
sums from 2609 - 2648

OFF "s" state 1
sums from 2654 - 2693

OFF "s" state 2
sum

OFF "s" state 4
sums from 3509 - 3548

ON "s" state 5
sums from 3554 - 3593

ON "s" state 6
sums from 3599 - 3638

seq:10
OFF "s" state 7
sums from 3644 - 3683

ON "s" state 0
sums from 3689 - 3728

OFF "s" state 1
sums from 3734 - 3773

OFF "s" state 2
sums from 3779 - 3818

ON "s" state 3
sums from 3824 - 3863

OFF "s" state 4
sums from 3869 - 3908

ON "s" state 5
sums from 3914 - 3953

ON "s" state 6
sums from 3959 - 3998

seq:11
OFF "s" state 7
sums from 4004 - 4043

ON "s" state 0
sums from 4049 - 4088

OFF "s" state 1
sums from 4094 - 4133

OFF "s" state 2
sums from 4139 - 4178

ON "s" state 3
sums from 4184 - 4223

OFF "s" state 4
sums from 4229 - 4268

ON "s" state 5
sums from 4274 - 4313

ON "s" state 6
sums from 4319 - 4358

seq:12
OFF "s" state 7
sums from 4364 - 4403

ON "s" state 0
sums from 4409 - 4448

OFF "s" state 1
sums from 4454 - 4493

OFF "s" state 2
sums from 4499 - 4538

ON "s" state 3
sums from 4544 - 4583

OFF "s" state 4
sums from 4589 - 4628

ON "s" state 5
s

OFF "s" state 4
sums from 3149 - 3188

ON "s" state 5
sums from 3194 - 3233

ON "s" state 6
sums from 3239 - 3278

seq:9
OFF "s" state 7
sums from 3284 - 3323

ON "s" state 0
sums from 3329 - 3368

OFF "s" state 1
sums from 3374 - 3413

OFF "s" state 2
sums from 3419 - 3458

ON "s" state 3
sums from 3464 - 3503

OFF "s" state 4
sums from 3509 - 3548

ON "s" state 5
sums from 3554 - 3593

ON "s" state 6
sums from 3599 - 3638

seq:10
OFF "s" state 7
sums from 3644 - 3683

ON "s" state 0
sums from 3689 - 3728

OFF "s" state 1
sums from 3734 - 3773

OFF "s" state 2
sums from 3779 - 3818

ON "s" state 3
sums from 3824 - 3863

OFF "s" state 4
sums from 3869 - 3908

ON "s" state 5
sums from 3914 - 3953

ON "s" state 6
sums from 3959 - 3998

seq:11
OFF "s" state 7
sums from 4004 - 4043

ON "s" state 0
sums from 4049 - 4088

OFF "s" state 1
sums from 4094 - 4133

OFF "s" state 2
sums from 4139 - 4178

ON "s" state 3
sums from 4184 - 4223

OFF "s" state 4
sums from 4229 - 4268

ON "s" state 5
su

In [17]:
print(len(ON_minus_sums[0][0]))
print((ON_minus_sums[10][0][0]))
print((ON_minus_sums[10][0][1]))
print((ON_plus_sums[10][0][0]))
print((ON_plus_sums[10][0][1]))

2
[  7.45022758  39.3060539   31.60210534 ... 127.24573612 143.32129248
  93.5409037 ]
[ -5.79925986 -15.54082391  24.88732904 ... 138.57757537 186.2198129
 141.10364453]
[ 32.57703823  -2.03201125 -15.81892974 ... 145.31333879 165.35360864
 136.99727979]
[-10.74293184 -22.31226584   7.04763139 ... 130.03282924 137.35363971
 150.448728  ]


In [15]:
ys = ys_basesub[0]
SFarr = sequence

#     tempadded_p = np.zeros((len(SFarr[0]), 8, len(ys[0])), dtype=np.float64)
temp_ON = np.zeros((len(SFarr[0]), len(ys[0])), dtype=np.float64)
temp_OFF = np.zeros((len(SFarr[0]), len(ys[0])), dtype=np.float64)

for seq in range(0, len(SFarr[0])): ## for every sequence
#         print('seq:' +str(SFarr[0][seq]))
#     print('seq:' +str(seq))
    for state in range(0, len(SFarr[1][0])): ## for every state in the sequence
#         print("states loop " + str(range(0, len(SFarr[1][0]))[0]) + ' - ' +  str(range(0, len(SFarr[1][0]))[-1]))
        s = SFarr[1][seq][state] ## try this to condense code. Basically, the state currently at
        if s==0 or s==3 or s==5 or s==6: ## these are ON states
#             print('"s" state '+str(s))
#             print('"state" ' +str(state) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])+1)))
#             print('actually from '+str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[0]) +
#             ' - ' +str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[-1]) + '\n')
            for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                temp_ON[seq] = np.add(temp_ON[seq],ys[p]) ## start with zeros, add to each iteratively
#                 print(p)
        if s==1 or s==2 or s==4 or s==7: ## these are OFF states
#             print('"s" state '+str(s))
#             print('"state" ' +str(state) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])+1)))
#             print('actually from '+str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[0]) +
#             ' - ' +str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[-1]) + '\n')
#                 print('OFF state: ' + str(s))
            for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
#                 print(p)
                temp_OFF[seq] = np.add(temp_OFF[seq],ys[p]) ## start with zeros, add to each iteratively
        


In[48]:

In [ ]:
start = time.time()

## OLD ASYM ####

 take off asym for now 08.09.24<br>
Asym = np.zeros((len(ON_OFF_sums), len(ON_OFF_sums[0][0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change)

In [ ]:
def asym(ON_OFF_arr):
    tempasym = np.zeros((len(ON_OFF_arr[0][0])), dtype=np.float64)
    for seq in range(len(ON_OFF_arr[0])): ## number of sequences
        asymform = ((ON_OFF_arr[seq][0]-ON_OFF_arr[seq][1]) / (ON_OFF_arr[seq][0]+ON_OFF_arr[seq][1]))
        tempasym = np.add(asymform,tempasym)
    normedasym = tempasym/len(ON_OFF_arr)
    return normedasym

 take off asym for now 08.09.24<br>
for i in range(len(ON_OFF_sums)):<br>
    Asym[i] = asym(ON_OFF_sums[i])

In [ ]:
end = time.time()
# print('calculate asymmetry time: ' + str(end-start)) 

## end OLD ####<br>
## NEW ####

In [43]:
asym_ch2 = np.zeros((len(ON_sums), len(ON_sums[0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change?)
asym_pm2 = np.zeros((len(ON_plus_sums),2, len(ON_plus_sums[0][0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change?)

In [44]:
def asym2(ON_arr, OFF_arr):
    tempasym = np.zeros((len(ON_arr[0])), dtype=np.float64)
    for seq in range(0, len(ON_arr)): ## number of sequences
#         print(seq)
        seqasym = ((ON_arr[seq]-OFF_arr[seq]) / (ON_arr[seq]+OFF_arr[seq]))
        tempasym = np.add(seqasym,tempasym)
    normedasym = tempasym/len(ON_arr)
    return normedasym

In [45]:
for i in range(len(ON_sums)):
    asym_ch2[i] = asym2(ON_sums[i], OFF_sums[i])
    asym_pm2[i][0] = asym2(ON_plus_sums[i][0], ON_plus_sums[i][1])
    asym_pm2[i][1] = asym2(ON_minus_sums[i][0], ON_minus_sums[i][1])
    # print('ch done')

In [46]:
print(asym_ch2[1][2100:2110])
print(asym_ch[1][2100:2110])

[ 4.51350923e-03  1.76767677e-03 -5.49848444e-03  2.09080142e-03
  2.30203919e-03 -5.87566348e-04 -4.86185945e-03 -4.55404051e-05
  3.88116865e-03  5.25332563e-04]
[ 4.51350923e-03  1.76767677e-03 -5.49848444e-03  2.09080142e-03
  2.30203919e-03 -5.87566348e-04 -4.86185945e-03 -4.55404051e-05
  3.88116865e-03  5.25332563e-04]


## end NEW ####

In [104]:
print(len(ON_sums))
print(len(ON_sums[0]))
print(len(ON_sums[0][0]))

13
13
9000


In [105]:
a = np.array([1,2,3,4,5])
b = np.array([11,12,13,14,15])
c = a+b
print(c)

[12 14 16 18 20]


In [106]:
testseqON = ON_sums[0][0]
print(type(testseqON))
print(len(testseqON))
print('testON')
print((testseqON[2100:2110]))
print('testOFF')
testseqOFF = OFF_sums[0][0]
print(len(testseqOFF))
print((testseqOFF[2100:2110]))
print()
print('testSUB')
testsub = testseqON-testseqOFF
print((testsub[2100:2110]))
print()
print('testADD')
testadd = testseqON+testseqOFF
print((testadd[2100:2110]))
print()
testdiv = testsub/testadd
print((testdiv[2100:2110]))

<class 'numpy.ndarray'>
9000
testON
[35452.92375922 36397.50980229 34571.25212892 36295.22234096
 38657.92286897 36252.27520379 38356.03278472 36705.21041393
 35699.0206249  36343.1469216 ]
testOFF
9000
[36561.52900406 37235.29691796 36190.16611324 36705.33785499
 36453.01766233 37066.91231337 36965.86601793 37041.52517143
 35650.21936505 35430.48283033]

testSUB
[-1108.60524484  -837.78711566 -1618.91398432  -410.11551403
  2204.90520664  -814.63710958  1390.1667668   -336.3147575
    48.80125985   912.66409127]

testADD
[72014.45276328 73632.80672025 70761.41824215 73000.56019595
 75110.94053131 73319.18751715 75321.89880265 73746.73558536
 71349.23998995 71773.62975192]

[-0.0153942  -0.01137791 -0.02287848 -0.00561798  0.02935531 -0.01111083
  0.01845634 -0.0045604   0.00068398  0.01271587]


In [18]:
# print(ON_sums)
# print(OFF_sums)

In [109]:
for seq in range(0, len(OFF_sums[1])):
    a = ON_sums[1][seq]+OFF_sums[1][seq]
    b = np.isfinite(a[seq])
    print(seq)
    print(np.all(b))

0
True
1
True
2
True
3
True
4
True
5
True
6
True
7
True
8
True
9
True
10
True
11
True
12
True


In [20]:
print(len(ON_sums))
print(len(ON_sums[0]))
print(len(ON_sums[0][0]))


12
13
9000


In [24]:
asym_seq = np.zeros((len(ON_sums),len(ON_sums[0]), len(ON_sums[0][0])), dtype=np.float64) ## 13 ch. 13 sequences, asym for each sequence
asym_seq_pm = np.zeros((len(ON_plus_sums),2, len(ON_plus_sums[0][0][0])), dtype=np.float64)
asym_ch = np.zeros((len(ON_sums), len(ON_sums[0][0])), dtype=np.float64) ## 13 ch. asym for each ch
asym_ch_pm = np.zeros((len(ON_plus_sums),2, len(ON_plus_sums[0][0][0])), dtype=np.float64)

def asym3(ON_arr, OFF_arr):
    tempasym = np.zeros((len(ON_arr), len(ON_arr[0])), dtype=np.float64)
    for seq in range(0, len(ON_arr)): ## number of sequences
#         print(seq)
        seqasym = ((ON_arr[seq]-OFF_arr[seq]) / (ON_arr[seq]+OFF_arr[seq]))
        tempasym[seq] = seqasym
#     normedasym = tempasym/len(ON_arr)
    return tempasym

for i in range(0, (len(ON_sums))):
    asym_seq[i] = asym3(ON_sums[i], OFF_sums[i])
print(len(asym_seq))
print(len(asym_seq[0]))
print(len(asym_seq[0][0]))

def chasym3(sequence_asyms):
    tempasym = np.zeros((len(sequence_asyms[0])), dtype=np.float64)
    for seq in range(0, len(sequence_asyms)): ## number of sequences
#         print(seq)
        tempasym = np.add(tempasym,sequence_asyms[seq])
    normedasym = tempasym/len(sequence_asyms)
    return normedasym

for i in range(0, (len(ON_sums))):
    print(i)
    asym_ch[i] = chasym3(asym_seq[i])
print(len(asym_ch))
print(len(asym_ch[0]))
# print(len(asym_ch[0][0]))

12
13
9000
0
1
2
3
4
5
6
7
8
9
10
11
12
9000


In [38]:
for i in range(13):
    print(asym_seq[1][i][2100:2110])
print(asym_ch[1][2100:2110])

[-0.01378357 -0.01232132  0.00693024  0.00696098 -0.00411387  0.00844475
  0.00375091  0.00876075 -0.0176183  -0.01358408]
[-0.00535022 -0.01496941 -0.00324674  0.00901851  0.00219262 -0.02203386
 -0.00249975 -0.00276314  0.01342373  0.01824402]
[-0.00284639  0.01378061  0.03376994  0.00601379 -0.00182133 -0.01088191
 -0.01770431  0.0039693  -0.02345179  0.00683798]
[-0.01092153 -0.01061586  0.00665349  0.00507361 -0.00748798 -0.01843978
 -0.01455302 -0.00656836  0.01688078  0.00559237]
[-0.01038564 -0.00207865  0.00274624 -0.00444893 -0.00751742  0.00691007
  0.01858317 -0.00777591 -0.03072655 -0.0143938 ]
[-0.01091814 -0.00644046 -0.03545888 -0.00694082 -0.01302527 -0.01260866
 -0.00821704 -0.02082586  0.01778585 -0.02744806]
[-0.00311721 -0.00819558  0.02243831  0.01339926 -0.00980873  0.01384114
 -0.01801435 -0.02514908  0.01529911  0.00609054]
[ 0.02101967  0.01424317 -0.0341947   0.01882657  0.03019377 -0.01244939
 -0.00087192  0.00057651  0.01523238  0.00538381]
[ 0.02426113  0.

In [42]:
print(np.average(asym_seq[1,:,2100]))

0.0045135092329460415


In[13]:

np.save(os.getcwd() + AddedPulseSavename, added_pulses)<br>
np.save(os.getcwd() + AsymSavename, Asym)

In [ ]:
np.save(os.getcwd() + AsymSavename, asym_ch)
np.save(os.getcwd() + '/processed_data/'+ 'xs_uncut', xs)

In [ ]:
fullend = time.time()
print('full processing time: ' + str(fullend-fullstart))  
print('finished ' + str(datetime.now())) 
print('\n')

## end of data processing ##

In[57]: